You are given a string s, and an array of pairs of indices in the string pairs where pairs[i] = [a, b] indicates 2 indices(0-indexed) of the string.

You can swap the characters at any pair of indices in the given pairs any number of times.

Return the lexicographically smallest string that s can be changed to after using the swaps.

 

Example 1:

Input: s = "dcab", pairs = [[0,3],[1,2]]
Output: "bacd"
Explaination: 
Swap s[0] and s[3], s = "bcad"
Swap s[1] and s[2], s = "bacd"
Example 2:

Input: s = "dcab", pairs = [[0,3],[1,2],[0,2]]
Output: "abcd"
Explaination: 
Swap s[0] and s[3], s = "bcad"
Swap s[0] and s[2], s = "acbd"
Swap s[1] and s[2], s = "abcd"
Example 3:

Input: s = "cba", pairs = [[0,1],[1,2]]
Output: "abc"
Explaination: 
Swap s[0] and s[1], s = "bca"
Swap s[1] and s[2], s = "bac"
Swap s[0] and s[1], s = "abc"
 

Constraints:

1 <= s.length <= 10^5
0 <= pairs.length <= 10^5
0 <= pairs[i][0], pairs[i][1] < s.length
s only contains lower case English letters.

In [ ]:
# brute force
# -  would be to loop over the pair until we can't make any more improvement.

def smallestStringWithSwaps(s: str, pairs: list[list[int]]) -> str:
    s = list(s)
    improved = True
    while improved:
        improved = False
        for i, j in pairs:
            if s[j] < s[i]:
                s[i], s[j] = s[j], s[i]
                improved = True
    return "".join(s)



#   * We loop over all `m` pairs → `O(m)` work.
# * In the **worst case**, we may keep swapping until the string becomes sorted.
# * That could take **up to exponential time (`O(n!)`)** because you might keep making small improvements step by step (like bubble sort but worse, since swaps are constrained by the graph of allowed pairs).

# So **Worst Case** = **Exponential** (`O(n! * m)` roughly).
# **Best Case** = `O(m)` (string already minimal, only one pass).

# * We store the string → `O(n)`




## 💡 Intuition with Example

`s = "dcab"`, pairs = `[(0,3), (1,2)]`

* Graph view:

  * `0 ↔ 3` (can swap freely between them).
  * `1 ↔ 2`.

So we have components:

* `{0,3}` → chars = `"d","b"` → sorted = `"b","d"`.
* `{1,2}` → chars = `"c","a"` → sorted = `"a","c"`.

Reassign sorted chars back to their indices:

* index 0 → `b`
* index 3 → `d`
* index 1 → `a`
* index 2 → `c`

Result: `"bacd"` ✅

---

## ⚙️ DSU Steps

1. Build DSU for `n = len(s)` indices.
2. For each pair `(i,j)`, call `union(i,j)` → merges indices into the same component.
3. Group indices by their root parent.
4. For each group:

   * Collect characters from those indices.
   * Sort them.
   * Reassign sorted characters back to those indices.



---

## ⏱️ Complexity

* Union-Find operations: `O(m α(n))` (≈ `O(m)`), where `m = len(pairs)`.
* Grouping + sorting:

  * Sorting all groups = `O(n log n)` (since each character is sorted once).
* **Total**: `O(n log n + m)`
* **Space**: `O(n + m)` for DSU + groups.

---

✅ **Intuition Recap**:

* DSU groups indices into connected components.
* Each group = “freely swappable zone.”
* Sort inside groups → gives smallest lexicographic arrangement.

---

Do you want me to also show you a **DFS/Graph version** (same idea but without DSU), so you can compare the two approaches?


In [ ]:

class DSU:
    def __init__(self, n):
        self.parent = list(range(n))
        self.size = [1] * n  # size of each set
    
    def find_parent(self, node):
        if self.parent[node] == node:
            return node
        self.parent[node] = self.find_parent(self.parent[node])  # path compression
        return self.parent[node]
    
    def union(self, x, y):
        u, v = self.find_parent(x), self.find_parent(y)
        if u == v:
            return
        # union by size.
        # We add low size groupd to the high size componenet.
        if self.size[u] < self.size[v]: # making u always bigger.
            u, v = v, u
        self.parent[v] = u
        self.size[u] += self.size[v]


class Solution:
    def smallestStringWithSwaps(self, s: str, pairs: list[list[int]]) -> str:
        n = len(s)
        dsu = DSU(n)

        # Step 1: Union pairs
        for i, j in pairs:
            dsu.union(i, j)

        # Step 2: Group indices by root parent
        from collections import defaultdict
        components = defaultdict(list)
        for i in range(n):
            root = dsu.find_parent(i)
            components[root].append(i)
        print(components)

        # Step 3: For each component, sort chars and place back
        res = list(s)
        for indices in components.values():
            chars = [s[i] for i in indices]

            # So you have to place the sorted character in the sorted index.
            indices.sort()
            chars.sort()
            for idx, ch in zip(indices, chars):
                res[idx] = ch

        return "".join(res)
    
# tc:
# sorting: O(n log n)
# DUS operation amorized to O(m)
# tc = O(m + n log n)

# sc:
# O(n + m)


- if we have (1,2) and (2,4) as a pair - can we acheive (1,4) swap ?
- yes since we can swap how many times we want.
- ex:    abcde
-        01234
- (1,2): acbde
-        01234
- (2,4): acedb
-        01234
- again do (1,2)
- (1,2): aecdb.    - this is equalent to (1,4)
-        01234



## Example

`s = "zbdcae"`
`pairs = [[0,3], [1,2], [2,4], [3,5]]`

---

### Step 1: String & Pairs

| Index | 0 | 1 | 2 | 3 | 4 | 5 |
| ----- | - | - | - | - | - | - |
| Char  | z | b | d | c | a | e |

Pairs:

* (0,3), (1,2), (2,4), (3,5)

---

### Step 2: Build DSU (Union-Find)

Perform unions:

* Union(0,3) → {0,3}
* Union(1,2) → {1,2}
* Union(2,4) → {1,2,4}
* Union(3,5) → {0,3,5}

---

### Step 3: Components (after all unions)

| Component (Root) | Indices | Chars   |
| ---------------- | ------- | ------- |
| Root A           | {0,3,5} | {z,c,e} |
| Root B           | {1,2,4} | {b,d,a} |

---

### Step 4: Sort Inside Each Component

| Component | Sorted Indices | Sorted Chars |
| --------- | -------------- | ------------ |
| {0,3,5}   | \[0,3,5]       | \[c,e,z]     |
| {1,2,4}   | \[1,2,4]       | \[a,b,d]     |

---

### Step 5: Place Sorted Chars Back

| Index | 0 | 1 | 2 | 3 | 4 | 5 |
| ----- | - | - | - | - | - | - |
| Char  | c | a | b | e | d | z |

Final Result:
**`"cabe dz"` → `"cabedz"`** ✅

---

## 💡 Intuition Recap

* DSU grouped swappable indices into **connected components**.
* Within each component → we can **reorder freely**, so sort for smallest lexicographic order.
* Rebuild the string.


